<a href="https://colab.research.google.com/github/AliBank/DenseNet201/blob/main/DenseNet201DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q keras
!pip install h5py pyyaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from tensorflow.keras.applications import DenseNet201 
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.layers import Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import RMSprop
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from glob import glob

In [4]:
train_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/train/images/"
test_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/test/images/"

In [5]:
numberOfClass=len(glob(train_path+"/*"))

In [6]:
print(numberOfClass)

30


In [7]:
dens= DenseNet201(include_top=False, weights='imagenet',input_shape=(224,224,3))

74836368/74836368 [==============================] - 1s 0us/step


In [ ]:
print(dens.summary())

In [9]:
print(type(dens))

<class 'keras.engine.functional.Functional'>


In [10]:
dens_layer_list= dens.layers
print(dens_layer_list)

[<keras.engine.input_layer.InputLayer object at 0x7f2fa9d6c610>, <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f2fa9d6cdf0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f30c6cfc250>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f30c6cd9dc0>, <keras.layers.core.activation.Activation object at 0x7f30c6cfc070>, <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x7f2fa94d1fa0>, <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7f2f7f6e1d90>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f2f7f6f59a0>, <keras.layers.core.activation.Activation object at 0x7f2f7f6f5cd0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f2f7f701d60>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f2f7f708160>, <keras.layers.core.activation.Activation object at 0x7f2f7f708910>, <keras.layers.convolutional.conv2d.Conv2D object at 0x7f2f7f6e1a90>, <keras

In [ ]:
dens.trainable=False
x =Flatten()(dens.output)
x =GlobalAveragePooling2D()(dens.output)
x= Dense(32, activation='relu')(x)
x= Dense(32, activation='relu')(x)
out=Dense(30,activation='softmax')(x)
model= Model(inputs=dens.inputs, outputs=out, name='DenseNet201')

print(len(model.layers))
for layer in model.layers:
  print(layer.name," ",layer.trainable)

for layer in model.layers[:10]:
  layer.trainable=False
model.summary()  


In [12]:
print(type(dens))

<class 'keras.engine.functional.Functional'>


In [13]:
from keras.optimizers.optimizer_v2.rmsprop import RMSProp
optim_1 = RMSProp(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=optim_1, metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
#train
train_data= ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224))
test_data= ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224))

Found 3601 images belonging to 30 classes.
Found 900 images belonging to 30 classes.


In [15]:
batch_size=50

In [24]:
from keras.callbacks import ModelCheckpoint,CSVLogger
filepath="/content/drive/MyDrive/saved_models/weights-improvement-{epoch:05d}.ckpt"
checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,save_freq=360, mode='max')
#log_csv =CSVLogger('my_logs.csv', separator=',', append=False)
callbacks_list =[checkpoint]

In [ ]:
#

In [25]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=10,
                         initial_epoch=0,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

<ipython-input-25-bad6f095120d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_data,


Epoch 1/10
72/72 [==============================] - 94s 1s/step - loss: 0.7547 - accuracy: 0.8069 - val_loss: 0.8978 - val_accuracy: 0.7587
Epoch 2/10
72/72 [==============================] - 87s 1s/step - loss: 0.6780 - accuracy: 0.8190 - val_loss: 0.8620 - val_accuracy: 0.7500
Epoch 3/10
72/72 [==============================] - 86s 1s/step - loss: 0.6526 - accuracy: 0.8218 - val_loss: 0.7618 - val_accuracy: 0.7760
Epoch 4/10
72/72 [==============================] - 88s 1s/step - loss: 0.6193 - accuracy: 0.8264 - val_loss: 0.7953 - val_accuracy: 0.7865
Epoch 5/10
71/72 [============================>.] - ETA: 0s - loss: 0.5478 - accuracy: 0.8485
Epoch 5: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00005.ckpt


72/72 [==============================] - 179s 3s/step - loss: 0.5478 - accuracy: 0.8488 - val_loss: 0.7270 - val_accuracy: 0.7865
Epoch 6/10
72/72 [==============================] - 107s 1s/step - loss: 0.5308 - accuracy: 0.8449 - val_loss: 0.6939 - val_accuracy: 0.7899
Epoch 7/10
72/72 [==============================] - 98s 1s/step - loss: 0.4883 - accuracy: 0.8607 - val_loss: 0.6633 - val_accuracy: 0.7969
Epoch 8/10
72/72 [==============================] - 96s 1s/step - loss: 0.4650 - accuracy: 0.8650 - val_loss: 0.6416 - val_accuracy: 0.7986
Epoch 9/10
72/72 [==============================] - 91s 1s/step - loss: 0.4543 - accuracy: 0.8689 - val_loss: 0.6108 - val_accuracy: 0.8142
Epoch 10/10
71/72 [============================>.] - ETA: 1s - loss: 0.4303 - accuracy: 0.8795
Epoch 10: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00010.ckpt


72/72 [==============================] - 172s 2s/step - loss: 0.4277 - accuracy: 0.8803 - val_loss: 0.5994 - val_accuracy: 0.8229


In [26]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00010.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=20,
                         initial_epoch=10,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00020.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=25,
                         initial_epoch=20,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00025.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=40,
                         initial_epoch=25,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00040.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=50,
                         initial_epoch=40,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00050.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=65,
                         initial_epoch=50,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00065.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=80,
                         initial_epoch=65,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

<ipython-input-27-17771f9ccd55>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_data,


Epoch 66/80
72/72 [==============================] - ETA: 0s - loss: 0.0752 - accuracy: 0.9852
Epoch 66: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00066.ckpt


72/72 [==============================] - 147s 2s/step - loss: 0.0752 - accuracy: 0.9852 - val_loss: 0.2920 - val_accuracy: 0.8941
Epoch 67/80
72/72 [==============================] - ETA: 0s - loss: 0.0721 - accuracy: 0.9852
Epoch 67: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00067.ckpt


72/72 [==============================] - 145s 2s/step - loss: 0.0721 - accuracy: 0.9852 - val_loss: 0.2684 - val_accuracy: 0.9045
Epoch 68/80
72/72 [==============================] - ETA: 0s - loss: 0.0682 - accuracy: 0.9870
Epoch 68: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00068.ckpt


72/72 [==============================] - 148s 2s/step - loss: 0.0682 - accuracy: 0.9870 - val_loss: 0.2950 - val_accuracy: 0.8941
Epoch 69/80
72/72 [==============================] - ETA: 0s - loss: 0.0693 - accuracy: 0.9857
Epoch 69: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00069.ckpt


...

72/72 [==============================] - 160s 2s/step - loss: 0.0556 - accuracy: 0.9900 - val_loss: 0.2497 - val_accuracy: 0.9097
Epoch 77/80
72/72 [==============================] - ETA: 0s - loss: 0.0559 - accuracy: 0.9909
Epoch 77: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00077.ckpt


72/72 [==============================] - 162s 2s/step - loss: 0.0559 - accuracy: 0.9909 - val_loss: 0.2398 - val_accuracy: 0.9253
Epoch 78/80
72/72 [==============================] - ETA: 0s - loss: 0.0538 - accuracy: 0.9904
Epoch 78: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00078.ckpt


72/72 [==============================] - 155s 2s/step - loss: 0.0538 - accuracy: 0.9904 - val_loss: 0.2417 - val_accuracy: 0.9167
Epoch 79/80
72/72 [==============================] - ETA: 0s - loss: 0.0472 - accuracy: 0.9926
Epoch 79: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00079.ckpt


72/72 [==============================] - 160s 2s/step - loss: 0.0472 - accuracy: 0.9926 - val_loss: 0.2820 - val_accuracy: 0.9080
Epoch 80/80
72/72 [==============================] - ETA: 0s - loss: 0.0511 - accuracy: 0.9891
Epoch 80: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00080.ckpt


72/72 [==============================] - 157s 2s/step - loss: 0.0511 - accuracy: 0.9891 - val_loss: 0.2396 - val_accuracy: 0.9201


In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00080.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         initial_epoch=80,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvement-00099.ckpt")

In [ ]:
##hist=model.fit_generator(train_data,
  ##                       steps_per_epoch=3600//batch_size,
    ##                     epochs=100,
      ##                   initial_epoch=99,
        ##                 validation_data= test_data,
          ##               validation_steps = 900//batch_size,
           ##              callbacks=callbacks_list)

<ipython-input-34-fb72b6c087c8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_data,


Epoch 100/100
72/72 [==============================] - ETA: 0s - loss: 0.0245 - accuracy: 0.9957
Epoch 100: saving model to /content/drive/MyDrive/saved_models/weights-improvement-00100.ckpt


72/72 [==============================] - 147s 2s/step - loss: 0.0245 - accuracy: 0.9957 - val_loss: 0.2147 - val_accuracy: 0.9201


In [ ]:
model.save_weights("BaklagilFold1RMSProplr0.0001Dens")